# Project Description 

## Analyzing Platooning Technologies and their benefits for transportation

The main objective of this project is to understand a traffic phenomena denominated "vehicle platooning" and design proper control strategies for a platoon of vehicles in order to optimize the traffic flow and reduce the fuel consumption. 

<img src="assets/img/truck-platooning.jpg" alt="drawing" width="600"/>

In order to do that we are going to make use of the package `vplatoon` that includes some basic models to deploy platooning systems including the implemenation of [`ACC/CACC`](https://en.wikipedia.org/wiki/Adaptive_cruise_control) systems. For more information about the source code of this package please go [here](https://github.com/research-licit/itstools/tree/main/itstools). Over there you will find a folder called `vplatoon` containing all the models in the submodules `carfollow`, `vehicles`, `demand`, `messages` etc. Some of these will be explored along the module and may help you to progress faster in the project.   


In [1]:
# Try this on google collab only
# !pip install -r https://raw.githubusercontent.com/research-licit/ITSProjects/main/requirements.txt

## Contents and structure 

* Exploration and modeling of vehicle dynamics and feedback systems 
* Basic control systems PID
* Implementation of a platoon via control systems 
* Compute emissions from trajectories 
* Understanding side effects of platoon (Stability) 

Let's begin by importing some plotting libraries that will help us with data visualization

In [2]:
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Understanding the basics 

For this application it is considered the simulation of [microscopic traffic models](https://en.wikipedia.org/wiki/Microscopic_traffic_flow_model) where *longitudinal position* follow a specified behavior defined as a function of  two main components, the *headway space* and the *speed differential*. The *car following* behavior describes the behavior of the vehicle in its longitudinal dynamics while the *lane change* behavior describes the behavior in the lateral position. 

In order to modify traffic behavior for a condition, the system is modeled via traffic model where a specific control algorithm regulates the vehicle speed or the inter vehicle distance.

#### Traffic model and the fundamental diagram

Before pursuing it is important to understand the value of the fundamental diagram. The fundamental diagram describe the relation ship between ***density*** or concentration of vehicles and the ***flow*** or speed of vehicles. The density is regularly denoted as $\kappa$ for this scenarios as `K`, in the meanwhile the flow is denoted by `Q`. We are going to consider the fact that the relationship follows a piecewise linear behavior as follows:


\begin{aligned}
Q(\kappa) = \begin{cases}
u\kappa\quad 0\leq\kappa\leq \kappa_c\\
w\left(\kappa_x-\kappa\right)\quad \kappa_c\leq\kappa\leq \kappa_x
\end{cases}
\end{aligned}


Let's define the number of vehicles and their initial positions, in order to determine the initial position let's find the minimum spacing. For the moment let's consider the following parameters. `W=5`, `K_X=0.2` and `U=20`. Let's trace the fundamental diagram for this case. We will make use of the `FundamentalDiagram` class in the `traffic` submodule

In [3]:
from itstools.vplatoon.traffic import FundamentalDiagram
from bokeh.plotting import show

f = FundamentalDiagram(w=5,u=25,k_x=0.2)
p = f.plot_diagram()
show(p)

We can easily establish the ***critical density*** which corresponds to the amount of vehicles at which there is maximum flow via

In [4]:
print(f"Capacity: {f.C}")
print(f"Critical density: {f.k_c}")
print(f"Maximum density: {f.k_x}")
print(f"Speed limit: {f.u}")
print(f"Congestion speed: {f.w}")

Capacity: 0.8333333333333334
Critical density: 0.03333333333333333
Maximum density: 0.2
Speed limit: 25
Congestion speed: 5


The critical density represent the minimum tolerable inter-vehicle distance **critical headway space** before vehicles start to decrease their speed. On the other hand when the value of density is maximum, and the flow is 0, the vehicles respect the **minimum headway space** that can be computed from the maximum value of density $\kappa_x$ via

\begin{aligned}
\kappa_x = \frac{N}{d} \approx \frac{1}{s_0}  \approx \frac{1}{l_{\text{avg}}+s_{\text{min}}} 
\end{aligned}

Where $l_{\text{avg}}$ is the average vehicle length and $s_{text{min}}$ represents the minimum tolerated headway spacing between two vehicles. 

By computing the case in the fundamental diagram

In [5]:
print(f"Minimum headway space: {1/f.k_x}")

Minimum headway space: 5.0


**Q1**: Supose a vehicle has an average length of $l_{\text{avg}}=4$ [m] and the minimum tolerable inter vehicle distance is $s_{\text{min}}=2.25$[m]. Consider the congestion wave speed $w=6.25$ [m/s] and a free flow speed of $u=25$ [m/s]. Compute and plot the fundamental diagram for this case and find the value of the capacity and minimum headway space:

<span style= "color:blue">Provide your answers below:</span>

#### Platooning behaviors and car following model 

Keep in mind the constants you have computed `K_X`, `W`, `U`, these parameters although apply for pure [car-following](https://en.wikipedia.org/wiki/Microscopic_traffic_flow_model) they will be useful to construct and understand the behavior of truck platoons. 

Let's consider the following figure.

<img src="assets/img/p2-cavs.png" alt="drawing" width="600"/>

For the sake of clarity, the following corresponds to the notations for variable description in the model. It is considered the vehicle position of a vehicle as $x_n$ and the headway space between a vehicle and its leader as $s_n = x_{n-1}-x_{n}$. The vehicle's speed and acceleration are defined as $v_n$,$a_n$ respectively. The operator $\Delta v_n = v_{n-1} - v_{n}$ refers to the difference between the leading vehicle and the following one. 

By introducing newton's law over one of the vehicles we can consider the following force set up and compute the accelertion given by the following expression: 

$$
a_n(t+T) = a_n(t) + T \frac{{u(t) - Mg\mu - {\scriptsize\frac{1}{2}}\rho C_d A v^2_n(t)}}{M} = a_n(t) + T g(C_d,v_n, u_n)
$$

Where $m$ corresponds to the truck mass, $C_d$ the drag coefficient, $A$ the frontal truck area, $\rho$ is the density of the air.  

Each one of the vehicles can follow the dynamics given by the following dynamic equation

\begin{aligned}
\frac{dx}{dt}\approx \frac{x_n(t+T) - x_n(t)}{T} \rightarrow x_n(t+T) &= x_n(t) + v_n(t)T\\
\frac{dv}{dt}\approx \frac{v_n(t+T) - v_n(t)}{T} \rightarrow v_n(t+T) &= v_n(t) + a_n(t) T\\
\frac{da}{dt}\approx \frac{a_n(t+T) - a_n(t)}{T} \rightarrow a_n(t+T) &= a_n(t) + T g(C_d,v_n,u_n)\\
\end{aligned}

*To implement the model a `class` object called `PlatoonDyn` has been implemented. The class intends to emulate these equations and keep track of the vehicle evolution. 

#### Parameters 

So far parameters in the model have been fixed although random scenarios can be also considered

| Parameter     | Value     | Units |
:--------------:|:---------:|:------:
$$M$$           | 0.5       |       |
$$A$$           | 10        | [m$^2$]
$$\rho$$        | 1.3       | [kg/m$^3$]
$${C_d}^\star$$ | 0.5 - 1.5 | 
$$\tau$$        | $$\frac{1}{wk_x}$$ | [s]
$$w$$           | $$6.25$$  | [m/s]
$$k_x$$         | $$0.16$$  | [veh/km]
$$u_i$$         | $$25$$    | [m/s]


$\star$: ${C_d}$ although is presented as parameter is a coefficient that changes according to the intervehicle distance. For the moment we have not considered the model but we will introduce this further in the system. 

Please follow step by step variable definition for more detail into simulations. Let define in this case the parameters `M`, `A`, `W`,`K_X` and import the class `VehiclePlatoon` from the submodule `vplatoon.carfollow`. In this time we will use simpler blocks but feel free to give it a look or ask any questions on how this class may work. 


#### Modeling a ACC Closed loop 

The objective in the upcoming will be to model the following closed loop:

<img src="assets/img/ctr-acc.png" alt="drawing" width="500"/>

This system is known as a feedback loop where $C(s)$ is denominated the **controller** $G(s)$ is denominated the **process** and the signals $r$, $u$, $y$ are denominated the **reference**, **control** and **output** signals respectively. The symbol in the $r$ signal represents a *step response* meaning the dynamic response of the system when the system changes abruptly from 0 to a particular value. 

First we are going to simulate this system via simple blocks and then use it to construct a platoon system. Consider just that the approximations of the derivative systems explained in the *Platoon behavior* section can be expressed as *transfer functions* 

<img src="assets/img/acc-det.png" alt="drawing" width="500"/>

$\tau$ represents the reaction time of the vehicle, $u$ the control signal (applied throttle), $a$ the real accelearation, $v$ the speed of the system. For the moment we are going to select a Proportional Integral Controller. For more details about the controller please see [here](https://fr.wikipedia.org/wiki/Régulateur_PID)

In [6]:
from itstools.vplatoon.vehicles import System,Integrator 
from itstools.vplatoon.pid import PID 
import numpy as np

# Declares each one of the systems
g1, g2, g3 = System(), System(), Integrator() # Declaring the system 1/(Tau s + 1) → System and the system 1/s→ Integrator

pid = PID(k_p=0.2, k_i=0.015,k_d=0) # PID Controller with initial gains

e_v = []
r_v = np.concatenate([np.zeros(10),np.concatenate([np.ones(600),0.5*np.ones(600)])]) # Creating reference signal

for ct in r_v:
    err = ct - g3.x[-1] 
    e_v.append(err)    
    uct = pid(err)
    g3(g2(g1(uct)))

The foor loop will run over all the samples of $r_v$ and apply each one of the entries to the system. It is possible to check the state of the system by checking the value $x$. Here the sytem allows to query the speed value.

In [7]:
from bokeh.plotting import figure

p = figure()
p.line(np.arange(len(g3.x)),g3.x, legend_label = "v")
p.line(np.arange(len(r_v)),r_v,color='red',legend_label = "r")
p.xaxis.axis_label="Time [s]"
p.yaxis.axis_label="Speed [m/s]"
p.legend.location = "top_right";
show(p)

The fact that the signal $v$ tries to pursuit the signal $r$ is the nature of the *controlled* system. In the further questions you will explore how this behavior can be changed in terms of the parameters.  

**Q2**: Let's consider the same exercise performed before, but now assume that the change in speed reference `r_v` (red signal) is not of value 1 but of value 5 instead. Perform a closed loop simulation and plot the results for the speed. In addition create a list called `u_v` for the signal `uctr`, append the values and plot also the result of this signal 

<span style= "color:blue">Provide your answers below:</span>

**Q3**: As you have observed the behavior was apriori pre established via fixed constants of the PID controller in particular `k_p`,`k_i`, `k_d`. For the moment let's fix the value of `k_d=0` and let's modify the value of `k_p` in the interval $[0.2,1]$ and the values of $k_i$ in the interval $[0.015,0.2]$. Select values in these intervals and observe how the dynamical response changes. Trace the behavior for the $v$ in time as well as the value of $u$ in time for different parameters

<span style= "color:blue">Provide your answers below:</span>

In [8]:
g1, g2, g3, g4 = System(), System(), Integrator(), Integrator()

# Declares each one of the systems
pid = PID(k_p=0.2, k_i=0.015,k_d=0)

e_x = []
r_x = np.concatenate([np.zeros(10),
                      np.cumsum(np.concatenate([np.ones(500),0.5*np.ones(500)]))])

for ct in r_x:
    err = ct - g4.x[-1]
    e_x.append(err)    
    uct = pid(err)
    g4(g3(g2(g1(uct))))

The foor loop will run over all the samples of $r_s$ and apply each one of the entries to the system. It is possible to check the state of the system by checking the value $x$. Here the sytem allows to query the position value.

In [9]:
p = figure()
p.line(np.arange(len(g4.x)),g4.x,legend_label = "x")
p.line(np.arange(len(r_x)),r_x,color='red',legend_label = "r")
p.xaxis.axis_label="Time [s]"
p.yaxis.axis_label="Position [m]"
p.legend.location = "top_left";
show(p)

**Q4**: Re run the simulation by storing the signal `uctr` in a variable `u_x`. Trace and observe the behavior of three variables in time. `e_x`, `u_x` and the corresponding speed. 

<span style= "color:blue">Provide your answers below:</span>

**Q5**: Again, as you have observed the behavior was apriori pre established via fixed constants of the PID controller in particular `k_p`,`k_i`, `k_d`. Again modify slightly the values of $k_p$ and $k_i$ and trace the signals $x$, $v$, $e_x$ and $u_x$ for a change in this case. What could be the potential effects of this control?

<span style= "color:blue">Provide your answers below:</span>


#### Building up a platoon of two vehicles 

In order to  connect a platoon of two vehicles in this case we need to create a chain of systems like the ones before studied. For this purpose we are going to use the same kind of building blocks. Note that in this case we need to provide a reference for *headway space* that needs to be transformed into position. Let's consider the following block diagram with two of the already simulated systems.


<img src="assets/img/cacc-det2.png" alt="drawing" width="700"/>

The idea is to put in cascade two controlled systems by computing the reference of the second one starting from the first one. Let's use again the same basic blocks 


In [10]:
# Class to enable connectivity betwen cars 

# Declare desired spacing 
HWY = 50

# Declare Systems 
g1, g2, g3, g4 = System(), System(), Integrator(HWY), Integrator(HWY)
g5, g6, g7, g8 = System(), System(), Integrator(0), Integrator(0)

# Declares Controllers
pid_x = PID(k_p=0.25, k_i=0.01,k_d=0)
pid_s = PID(k_p=0.25, k_i=0.01,k_d=0)

# Error signals 
e_x = []
e_s = []

# Compute reference 
r0_x = np.concatenate([np.zeros(10),
                      np.cumsum(np.concatenate([np.ones(500),0.5*np.ones(500)]))])+HWY # Starts from 50

r1_s = - HWY * np.ones(len(r0_x))



for r_x, r_s in zip(r0_x, r1_s):
    
    # Position error 
    err_x = r_x - g4.x[-1]
    e_x.append(err_x)
    
    s = g8.x[-1]-g4.x[-1]
    
    # Space error
    err_s = r_s - s
    e_s.append(err_s)
    
    # Control compute 
    ctr0 = pid_x(err_x) # Leader
    ctr1 = pid_s(err_s) # Follower 
    
    # Dynamic evolution 
    g4(g3(g2(g1(ctr0))))    
    g8(g7(g6(g5(ctr1))))
    

## PLOTTING
    
p = figure()
p.line(np.arange(len(r0_x)),r0_x, color = "darkblue" ,line_dash='dotted',legend_label = "r0")
p.line(np.arange(len(g4.x)),g4.x,legend_label = "x0")

p.line(np.arange(len(r1_s)),r0_x+r1_s, color = "darkred" ,line_dash='dotted',legend_label = "r1")
p.line(np.arange(len(g8.x)),g8.x, color="red" ,legend_label = "x1")

p.xaxis.axis_label="Time [s]"
p.yaxis.axis_label="Position [m]"
p.legend.location = "top_left";
show(p)
        

**Q6**: Check the speed behavior for each one of the vehicles in the platoon, trace the signals $x$, $v$, $s$, $e_x$, $e_s$, $u_x$ and $u_s$ this case.  After observing the effects onte the follower consider modifying the gains $k_p$ to $0.2$ and $k_i$ to $0.015$. Trace again this curves and check the dynamic behavior. 

<span style= "color:blue">Provide your answers below:</span>

**Q7**: Regularly the behavior of platoon should adapt to traffic manuevers, for the platoon of $2$ vehicles consider modyifing the policy of *headway space* via a linear function so that the amount of gap of vehicles changes from the current value `HWY=50` to a new value of `HWY = 20`. Extract the profiles for $x$, $v$, $s$. 

<span style= "color:blue">Provide your answers below:</span>


**Q8**: Consider creating a platoon of 3 vehicles this time, extract in this case the trajectory plots. 

<span style= "color:blue">Provide your answers below:</span>

#### Compute Total Travel Time 

To compute the total travel time we consider the time taken from the position where the leader starts its trip until a specified distance in kilometers `d`. We make use of `pandas` to filter data, the module `datetime` to find the time diference and the method `apply` to compute the values for all vehicles in the network. Examine the code down below for details on this computation. 

In [11]:
import datetime  as dt 
import pandas as pd 

time = np.linspace(0,len(g4.x),len(g4.x))
X = np.array([g4.x, g8.x]).T

# Creation of Pandas dataframe 
df_x = pd.DataFrame(X)

now = dt.datetime(2019,12,17,13,0) # This is to set a special day for synchronous purposes 
delta = dt.timedelta(seconds=1)
time_vector = [now+n*delta for n in range(int(np.max(time)))]
df_x.index = time_vector 

# Compute the total travel time from 0 to 700 m
def find_travel_time(df,max_dist = 700):
    time_entry  = min(df[df>df_x.iloc[0,0]].index) # minimum time after leader entrance 
    time_exit = max(df[df<max_dist].index) # maximum time before exit
    travel_time = time_exit-time_entry
    return travel_time.total_seconds()

ttt = df_x.apply(find_travel_time)
tt_mean,tt_var = np.mean(ttt),np.std(ttt) 
ttt = sum(ttt)

print(f"Mean Travel time: {tt_mean}, Variance Travel time: {tt_var}, Total Traveltime:{ttt}")

Mean Travel time: 821.5, Variance Travel time: 25.5, Total Traveltime:1643.0


**Q9**: For the case of the platoon of 3 vehicles compute again the *total travel time* and plot the *headway space* in terms of time. What can you conlude when the number of vehicles tend to augment 

<span style= "color:blue">Provide your answers below:</span>

#### Computing emissions and side effects 

When checking emissions we can refer multiple models, in this case we are going to make use of the model given by [Treiber, 2014](http://traffic-flow-dynamics.org). The function to compute instantaneous $CO_2$ emissions is given by: 

$$\dot{CO}_2 =  2.3 \frac{100000}{C_{\text{spec}}} \frac{P(v,\dot{v})}{v}$$

where 

$$P(v,\dot{v}) = max(P_0+ v F(v,\dot{v}),0)$$

$$F(v,\dot{v}) = m\dot{v} + (\mu+\phi)mg + \frac{1}{2}\rho \bar{c_d} A v^2$$

Some of these parameters are: 

| Parameter     | Value     | Units |
:--------------:|:---------:|:------:
$$m$$           | 1500      | [kg]
$$P_0$$         | 3         | [kW]
$$C_{\text{spec}}$$| 300    | [ml/kWh]
$$\rho$$        | 1.3       | [kg/m$^3$]
$$\mu$$         | 0.02      | [veh/km]
$$\bar{c_d}$$         | 0.3       | 
$$A$$           | 2         | [m$^2$]

$\dot{v}$ denote the vehicle acceleration and $v$ denote the vehicle speed

**Q10**: Design a function that accepts the matrices `V` and `A` and returns the emissions of each vehicle in time. Compute the cumulated emissions in time for all vehicles and compare to the case of `V_CAV`,  `A_CAV`. 

<span style= "color:blue">Provide your answers below:</span>


## Project 03

* Create and simulate a platoon of 5 vehicles by considering uniform spacing among them.
* Study the behavior of the average `headway space` with particular values of the PID. You may use the same as in this notebook. 
* Consider varying the constants `k_p` and `k_i` from their current values until some instability small instability es reached. Which are the consequences for the last vehicle in the platoon. 
* Compute the value of flow `Q` according to the fundamental diagram. 
* Determine the net effect on estimated `CO_2` emissions due to the aplication of different controllers?
* If the coefficient `c_d` is now a function of the spacing like 

    $$c_d = \frac{\bar{c_d}}{1+e^{-x/2}}$$

  What are effects on the `CO_2` emissions? 


A. Ladino